In [1]:
import numpy as np 
import torch 
from collections import defaultdict
import json 
import pathlib 
import sys



In [2]:
annotations = json.load(open("/brtx/603-nvme2/estengel/annotator_uncertainty/vqa/dev_from_mturk/annotations.json"))['annotations']
questions = json.load(open("/brtx/603-nvme2/estengel/annotator_uncertainty/vqa/dev_from_mturk/questions.json"))['questions']

In [3]:
# get the clusters from annotations 
def get_annotator_clusters(questions, annotations): 
    anns_by_qid = defaultdict(list)
    for quest, ann in zip(questions, annotations):

        qid, i = quest['question_id'].split("_")
        anns_by_qid[qid].append((quest, ann))

    clusters_by_qid = {}
    for qid, list_of_qas in anns_by_qid.items():
        clusters = defaultdict(list)
        for quest, ann in list_of_qas:
            rewritten = quest['new_question']
            answer = ann['answers'][0]['answer']
            answer_id = ann['answers'][0]['mturk_id']
            cluster_dict = {"answer": answer, "id": answer_id} 
            clusters[rewritten].append(cluster_dict)
        clusters_by_qid[qid] = clusters
    return clusters_by_qid



# get the clusters from kmeans preprocessing
def get_preprocessed_clusters(questions, annotations): 
    anns_by_qid = defaultdict(list)
    for quest, ann in zip(questions, annotations):

        qid, i = quest['question_id'].split("_")
        anns_by_qid[qid].append((quest, ann))

    clusters_by_qid = {}
    for qid, list_of_qas in anns_by_qid.items():
        clusters = defaultdict(list)
        for quest, ann in list_of_qas:
            answer = ann['answers'][0]['answer']
            answer_id = ann['answers'][0]['mturk_id']
            id_key, answer_id_suffix = answer_id.split(".")
            cluster_dict = {"answer": answer, "id": answer_id} 
            clusters[id_key].append(cluster_dict)
        clusters_by_qid[qid] = clusters
    return clusters_by_qid


In [ ]:
# get the clusters from predictions 
def get_prediction_clusters(predictions_jsonl, score_method='bert'):
    anns_by_qid = defaultdict(list)
    for quest, ann in zip(questions, annotations):
        qid, i = quest['question_id'].split("_")
        anns_by_qid[qid].append((quest, ann))

    vectors_by_qid = defaultdict(list)
    answers_by_qid = defaultdict(list)

    # print(anns_by_qid)
    for qid, list_of_qas in anns_by_qid.items():
        image_id = list_of_qas[0][0]['image_id']
        for quest, ann in list_of_qas:
            qid, i = quest['question_id'].split("_")
            answer = ann['answers'][0]['answer']
            answer_id = ann['answers'][0]['mturk_id']
            strings_by_qid[qid].append((, int(i)))
            answers_by_qid[qid].append({"answer": answer, "id": answer_id})

    clusters_by_qid = {}
    for qid, strings in strings_by_qid.items():
        clusters = cluster_strings(strings, score_method=score_method)

        clusters = {k: [answers_by_qid[qid][idx] for idx in v ] for k, v in clusters.items()}
        clusters_by_qid[qid] = clusters

    return clusters_by_qid